<a href="https://colab.research.google.com/github/pri188/Capstone-3---Classify-Image-if-it-is-GenAI-or-Real/blob/main/Is_GenAI_Image_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install Dependencies

In [1]:
# %pip install torch==2.5.0 torchvision ftfy regex tqdm git+https://github.com/openai/CLIP.git

# Read images and generate CLIP embeddings

In [2]:
import torch
import clip
from PIL import Image

device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-B/32", device=device)

def get_image_embedddings(image_path):
  image = preprocess(Image.open(image_path)).unsqueeze(0).to(device)
  image_features = model.encode_image(image).detach().numpy().tolist()
  return image_features

In [3]:
from os import listdir
from os.path import isfile, join
import numpy as np

categories = ['real', 'genai']
embeddings = []
for category in categories:
  category_path = f'genai_data/{category}/'
  image_names = [f for f in listdir(category_path) if isfile(join(category_path, f))]
  print(image_names)
  for image_name in image_names:
    image_features = get_image_embedddings(f'{category_path}{image_name}')
    label = 0 if category == 'real' else 1
    image_features.append(image_name)
    image_features.append(label)
    embeddings.append(image_features)

['Saltwater Crocodile.jpeg', 'Zebra.jpeg', 'house.jpg', 'Mango_4.jpg', 'Lion.jpeg', 'Elephant.jpg', 'Baby.jpg', 'Eiffel Tower.jpeg', 'Real Apple.jpg', 'table_grapes_grapes_fruit.jpg', 'Parrot.jpg']
['Parrot.jpeg', 'Lion.jpg', 'Apple.jpg', 'baby.jpeg', 'Zebra.jpg', 'Mango_4.jpg', 'Lion.jpeg', 'table grapes.jpg', 'Elephant.jpg', 'Baby.jpg', 'Eiffel Tower.jpeg', 'Real Apple.jpg']


In [4]:
import pandas as pd

df = pd.DataFrame(embeddings, columns=['embeddings', 'image_name', 'label'])
df.head()

,embeddings,image_name,label
0,"[0.3451813757419586, -0.5790499448776245, -0.2...",Saltwater Crocodile.jpeg,0
1,"[0.30487000942230225, -0.22138716280460358, -0...",Zebra.jpeg,0
2,"[-0.04642140120267868, 0.2565220892429352, 0.1...",house.jpg,0
3,"[0.2503420412540436, 0.28583136200904846, -0.0...",Mango_4.jpg,0
4,"[0.03871428593993187, -0.39236465096473694, -0...",Lion.jpeg,0


# Convert labels and features to train test set

In [17]:
from sklearn.model_selection import train_test_split


total_repeatation = 1000
X = df.drop('label', axis=1)
y = df['label']

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train = pd.concat([X_train] * total_repeatation, ignore_index=True)
X_train = np.array(X_train['embeddings'].to_list(), dtype=np.float32)
X_train = torch.from_numpy(X_train)

X_test = pd.concat([X_test] * total_repeatation, ignore_index=True)
X_test = np.array(X_test['embeddings'].to_list(), dtype=np.float32)
X_test = torch.from_numpy(X_test)

y_train = np.array(y_train.to_list(), dtype=np.float32)
y_train = np.tile(y_train, total_repeatation) # Repeat same data n times
y_train = torch.tensor(y_train, dtype=torch.float32).reshape(-1, 1)

y_test = np.array(y_test.to_list(), dtype=np.float32)
y_test = np.tile(y_test, total_repeatation) # Repeat same data n times
y_test = torch.tensor(y_test, dtype=torch.float32).reshape(-1, 1)

len(X_train), len(X_test), len(y_train), len(y_test)

# Train a model using pytorch Neural Network

In [19]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_classification
import numpy as np


# Generate random data for testing (do not use)
'''X, y = make_classification(n_samples=1000, n_features=20, n_informative=15, n_redundant=5, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Convert data to tensors
X_train = torch.tensor(X_train, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32).reshape(-1, 1)
y_test = torch.tensor(y_test, dtype=torch.float32).reshape(-1, 1)'''

# Define the model
class BinaryClassifier(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(BinaryClassifier, self).__init__()
        self.linear1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.linear2 = nn.Linear(hidden_size, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        out = self.linear1(x)
        out = self.relu(out)
        out = self.linear2(out)
        out = self.sigmoid(out)
        return out

# Instantiate the model
input_size = X_train.shape[1]
hidden_size = 256
model = BinaryClassifier(input_size, hidden_size)

# Define loss function and optimizer
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

# Train the model
epochs = 100
for epoch in range(epochs):
    # Forward pass
    y_pred = model(X_train)
    loss = criterion(y_pred, y_train)

    # Backward and optimize
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if (epoch+1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}')

Epoch [10/100], Loss: 0.2335
Epoch [20/100], Loss: 0.2315
Epoch [30/100], Loss: 0.2322
Epoch [40/100], Loss: 0.2311
Epoch [50/100], Loss: 0.2312
Epoch [60/100], Loss: 0.2311
Epoch [70/100], Loss: 0.2311
Epoch [80/100], Loss: 0.2311
Epoch [90/100], Loss: 0.2311
Epoch [100/100], Loss: 0.2311


# evaluate the model

In [20]:
with torch.no_grad():
    y_pred = model(X_test)
    y_pred_class = (y_pred.round())
    accuracy = (y_pred_class == y_test).float().mean()
    print(f'Accuracy: {accuracy:.4f}')

Accuracy: 0.2000
